In [1]:
import numpy as np     #只需要下载numpy库即可
import random
import GridWorld_v2
import time
from IPython.display import clear_output

In [2]:
rows = 5      #记得行数和列数这里要同步改
columns = 5

# gridworld = GridWorld_v2.GridWorld_v2(rows=rows, columns=columns, forbiddenAreaNums=8, targetNums=2, seed = 52,forbiddenAreaScore=-10)
# gridworld = GridWorld_v2.GridWorld_v2(desc = [".#",".T"])             #赵老师4-1的例子
# gridworld = GridWorld_v2.GridWorld_v2(desc = ["##.T","...#","...."])  #随便弄的例子
gridworld = GridWorld_v2.GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = [".....",".##..","..#..",".#T#.",".#..."]) 
#gridworld = GridWorld_v2(forbiddenAreaScore=-10, score=1,desc = ["T."]) 
gridworld.show()


value = np.zeros(rows*columns)       #初始化可以任意，也可以全0
qtable = np.zeros((rows*columns,5))  #初始化，这里主要是初始化维数，里面的内容会被覆盖所以无所谓


⬜️⬜️⬜️⬜️⬜️
⬜️🚫🚫⬜️⬜️
⬜️⬜️🚫⬜️⬜️
⬜️🚫✅🚫⬜️
⬜️🚫⬜️⬜️⬜️


In [18]:
def feature_function(state, action):
    x = state // 5
    y = state %  5
    features = [x,y,x*y,action]
    return np.array(features)

In [19]:
def q_function_approximation(state, action, weights):
    features = feature_function(state, action)
    q_value = np.dot(weights, features)
    return q_value

In [5]:
import math

In [20]:
def QLearning_Off_Policy(gridworld:GridWorld_v2.GridWorld_v2, grad, w,TrajectoryPoolSize = 50,gamma = 0.99,trajectorySteps=-1, learning_rate=0.001, num_episodes=600)->GridWorld_v2.GridWorld_v2:
    """
    这是最基础的SARSA算法

    Parameters:
    trajectorySteps (int): 寻路的轨迹长度，如果是-1，则为寻到目的则停止，否则参数即为trajectory长度
    learning_rate (float): 学习率，用于调节TD-target
    epsilon (float): epsilon-greedy的核心参数，0~1的浮点数，其中1则表示当前state所有决策概率一样，0则表示决策没有任何的随机性
    num_episodes (int): 表示模型迭代次数

    Returns:
    GridWorld_v2.GridWorld_v2: 把模型返回回去
    """
    
    state_value = np.zeros((rows * columns))
    action_value = np.zeros((rows * columns, 5))
    policy = np.eye(5)[np.random.randint(0,5,size=(rows*columns))] 
    # behavior_policy = 1000
    behavior_policy = np.ones((rows * columns, 5)) * 0.2
    epsilon = 0.5
    
    initState=10

    TrajectoryPool = []
    TrajectoryPoolSize = 50
    for i in range(TrajectoryPoolSize):
        if trajectorySteps==-1:
            stop_when_reach_target = True
            # 注意这里的返回值是大小为(trajectorySteps+1)的元组列表，因为把第一个动作也加入进去了
            # 这里改成behavior_policy
            Trajectory = gridworld.getTrajectoryScore(nowState=initState, 
                                                      action=random.randint(0,4) , 
                                                      policy=behavior_policy, 
                                                      steps=trajectorySteps, 
                                                      stop_when_reach_target=True)
        TrajectoryPool.append(Trajectory)
        
    for episode in range(num_episodes):
        #清除输出，可以更好的展示策略
        
        #cnt数组用来检查每个state有多少次访问
        cnt = [0 for i in range(25)]
        
        for Trajectory in TrajectoryPool:
            steps = len(Trajectory) - 1
            for k in range(steps,-1,-1):
                #State，Action，Reward，NextState，NextAction
                tmpstate, tmpaction, tmpscore, nextState, nextAction  = Trajectory[k]
                cnt[tmpstate] += 1
                nowX, nowY, nextX, nextY = tmpstate//5,tmpstate%5,nextState//5,nextState%5

                
                nextMax = max(grad(nextX,nextY,i) @ w for i in range(5))
                nowMax = max(grad(nowX,nowY,i) @ w for i in range(5))
                TD_error = (tmpscore + gamma * nextMax) - grad(nowX,nowY,tmpaction) @ w
                w += learning_rate * TD_error * grad(nowX, nowY,tmpaction)

        # policy improvement
        # policy = np.eye(5)[np.argmax(action_value,axis=1)]  #qtable的最优值作为更新策略，并用独热码来表示
        tmp_policy = []
        for i in range(25):
            x = i//5
            y = i%5
            maxx = grad(x,y,0)@w
            index = 0
            for action in range(5):
                # print(grad(x,y,action)@w)
                if grad(x,y,action)@w>maxx:
                    maxx = grad(x,y,action)@w
                    index = action
            tmp_policy.append(index)
            
        # print(tmp_policy)
        policy = np.eye(5)[tmp_policy]  #qtable的最优值作为更新策略，并用独热码来表示
        
        if episode % 10 == 0:
            print("episode",f"{episode}/{num_episodes}")
            print((TD_error * grad(2,0,tmpaction)).mean())
            gridworld.showPolicy(policy)

    return action_value
    

In [27]:
grad = lambda x,y,z: np.array([1, x, y, z, x*x, y*y, z*z, x*y, x*z, y*z, x*x*y, x*y*y,z**3] )
np.random.seed(1)
w = np.random.rand(13)
QLearning_Off_Policy(gridworld, grad=grad, w=w, learning_rate=0.000001)

episode 0/600
-2.250211183410439
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄
episode 10/600
-636.5047210464556
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄
episode 20/600
-584333.328970357
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄
episode 30/600
-539949388.7002265
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄
episode 40/600
-498945040120.8951
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄
episode 50/600
-461054625699621.44
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄🔄🔄🔄
🔄🔄✅🔄🔄
🔄🔄🔄🔄🔄


KeyboardInterrupt: 